In [1]:
%%html
<style> body { font-family: "Times New Roman"} </style>   

In [2]:
import ast
import dask
import dask.dataframe as dd
import pandas as pd
from dask.distributed import Client

/home/mmora/anaconda3/lib/python3.7/site-packages/dask/config.py:131: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/mmora/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [4]:
#client.close()
client = Client(processes=False, memory_limit='8GB')
client

Client Scheduler: inproc://192.168.0.15/11725/1,Cluster Workers: 1 Cores: 12 Memory: 8.00 GB


# 1. Carga de datos

In [3]:
df = pd.read_csv('DATA_CRAN/CRAN_DATA_20191231.csv', sep='|', low_memory=False)
df.shape

(91321, 39)

Elimina filas donde la fecha de publicación se nula

In [5]:
versionOfPackages = df.dropna(subset=['DatePublication']).copy()
versionOfPackages.set_index(["Year/Publication", "Package"], inplace=True)
versionOfPackages.sort_values('DatePublication', ascending=False, inplace=True)
versionOfPackages.head()

Author  \
Year/Publication Package                                                           
2019             slurmR        George Vega Yon [aut, cre] (<https://orcid.org...   
                 workflows              Davis Vaughan [aut, cre],\nRStudio [cph]   
                 multilinguer  Chanyub Park [aut, cre] (<https://orcid.org/00...   
                 sbw           Jose R. Zubizarreta [aut, cre],\nYige Li [aut]...   
                 uavRst        Chris Reudenbach [cre, aut],\nHanna Meyer [aut...   

                                                                       Authors@R  \
Year/Publication Package                                                           
2019             slurmR        c( person("George", "Vega Yon", email = "g.veg...   
                 workflows     \nc(person(given = "Davis",\nfamily = "Vaughan...   
                 multilinguer  person(given = "Chanyub",\nfamily = "Park",\nr...   
                 sbw           c(person("Jose R.", "Zubizarreta", role = c("a...   
                 uavRst        c(person("Chris", "Reudenbach", email = "reude...   

                                                                     BugReports  \
Year/Publication Package                                                          
2019             slurmR            https://github.com/USCbiostats/slurmR/issues   
                 workflows       https://github.com/tidymodels/workflows/issues   
                 multilinguer  https://github.com/mrchypark/multilinguer/issues   
                 sbw                                                        NaN   
                 uavRst                 https://github.com/gisma/uavRst/issues/   

                                     Date         Date/Publication  \
Year/Publication Package                                             
2019             slurmR               NaN  2019-12-31 02:00:02 UTC   
                 workflows            NaN  2019-12-30 14:00:02 UTC   
                 multilinguer         NaN  2019-12-30 17:00:02 UTC   
                 sbw           2019-12-06  2019-12-30 15:20:02 UTC   
                 uavRst        2019-12-18  2019-12-30 14:50:03 UTC   

                                                           Depends  \
Year/Publication Package                                             
2019             slurmR                          ['R', 'parallel']   
                 workflows                                   ['R']   
                 multilinguer                                   []   
                 sbw           ['R', 'Matrix', 'quadprog', 'slam']   
                 uavRst                                      ['R']   

                                                                     Description  \
Year/Publication Package                                                           
2019             slurmR        'Slurm', Simple Linux Utility for Resource Man...   
                 workflows     Managing both a 'parsnip' model and a preproce...   
                 multilinguer  Provides install functions of other languages\...   
                 sbw           Weights of minimum variance that approximately...   
                 uavRst        Support the analysis of drone derived imagery ...   

                              Encoding  \
Year/Publication Package                 
2019             slurmR          UTF-8   
                 workflows       UTF-8   
                 multilinguer    UTF-8   
                 sbw               NaN   
                 uavRst          UTF-8   

                                                                         Imports  \
Year/Publication Package                                                           
2019             slurmR                                                ['utils']   
                 workflows     ['cli', 'ellipsis', 'generics', 'glue', 'hardh...   
                 multilinguer  ['sys', 'fs', 'usethis', 'zip', 'askpass', 'xm...   
                 sbw        

In [6]:
def last_version_package_by_year(df):
    return df.iloc[[0]]

# Agrupa por año y paquete para obtener la última versión de paquete públicada en ese año
versionByYear = versionOfPackages.groupby(level=[0,1], group_keys=False).apply(last_version_package_by_year)
versionByYear.reset_index(inplace=True)
versionByYear.shape

(43161, 39)

In [7]:
df_T = versionByYear[['Package', 'Version', 'Imports', 'Year/Publication', 'Year_Month/Publication']].copy()
df_T['Imports'] = df_T["Imports"].apply(ast.literal_eval)
df_T['SizeImports'] = df_T["Imports"].str.len()
df_T.head()

,Package,Version,Imports,Year/Publication,Year_Month/Publication,SizeImports
0,leaps,NaN,[],1997,1997-12,0
1,akima,0.2-3,[],1998,1998-08,0
2,logspline,NaN,[],1998,1998-11,0
3,mda,0.1-1,[],1998,1998-04,0
4,oz,1.0-3,[],1998,1998-02,0


Tenemos versiones de paquetes que no tienen paquetes importados, a continuación eliminamos estas filas

In [8]:
df_T = df_T[(df_T['SizeImports'] != 0)]
df_T.shape

(23753, 6)

In [9]:
df_T.head()

,Package,Version,Imports,Year/Publication,Year_Month/Publication,SizeImports
553,Matrix,0.8-21,"[graphics, stats, lattice, grid]",2004,2004-12,4
614,clue,0.1-0,[cluster],2004,2004-12,1
634,e1071,1.5-5,"[graphics, stats]",2004,2004-12,2
659,foreign,0.8-4,"[stats, methods]",2004,2004-12,2
696,lattice,0.11-2,"[grid, grDevices, graphics, stats]",2004,2004-12,4


Dividimos en conjuntos de datos por año, para los últimos diez años

In [10]:
df_T_2010 = df_T[(pd.to_numeric(df_T['Year/Publication']) == 2010)].copy()
df_T_2010.reset_index(inplace=True)
df_T_2010.drop(['SizeImports','index'], axis=1, inplace=True)
df_T_2010.shape

(178, 5)

In [11]:
df_T_2011 = df_T[(pd.to_numeric(df_T['Year/Publication']) == 2011)].copy()
df_T_2011.reset_index(inplace=True)
df_T_2011.drop(['SizeImports','index'], axis=1, inplace=True)
df_T_2011.shape

(259, 5)

In [12]:
df_T_2012 = df_T[(pd.to_numeric(df_T['Year/Publication']) == 2012)].copy()
df_T_2012.reset_index(inplace=True)
df_T_2012.drop(['SizeImports','index'], axis=1, inplace=True)
df_T_2012.shape

(405, 5)

In [13]:
df_T_2013 = df_T[(pd.to_numeric(df_T['Year/Publication']) == 2013)].copy()
df_T_2013.reset_index(inplace=True)
df_T_2013.drop(['SizeImports','index'], axis=1, inplace=True)
df_T_2013.shape

(744, 5)

In [14]:
df_T_2014 = df_T[(pd.to_numeric(df_T['Year/Publication']) == 2014)].copy()
df_T_2014.reset_index(inplace=True)
df_T_2014.drop(['SizeImports','index'], axis=1, inplace=True)
df_T_2014.shape

(1476, 5)

In [25]:
df_T_2015 = df_T[(pd.to_numeric(df_T['Year/Publication']) == 2015)].copy()
df_T_2015.reset_index(inplace=True)
df_T_2015.drop(['SizeImports','index'], axis=1, inplace=True)
df_T_2015.shape

(2433, 5)

In [26]:
df_T_2016 = df_T[(pd.to_numeric(df_T['Year/Publication']) == 2016)].copy()
df_T_2016.reset_index(inplace=True)
df_T_2016.drop(['SizeImports','index'], axis=1, inplace=True)
df_T_2016.shape

(3319, 5)

In [27]:
df_T_2017 = df_T[(pd.to_numeric(df_T['Year/Publication']) == 2017)].copy()
df_T_2017.reset_index(inplace=True)
df_T_2017.drop(['SizeImports','index'], axis=1, inplace=True)
df_T_2017.shape

(4100, 5)

In [28]:
df_T_2018 = df_T[(pd.to_numeric(df_T['Year/Publication']) == 2018)].copy()
df_T_2018.reset_index(inplace=True)
df_T_2018.drop(['SizeImports','index'], axis=1, inplace=True)
df_T_2018.shape

(4946, 5)

In [29]:
df_T_2019 = df_T[(pd.to_numeric(df_T['Year/Publication']) == 2019)].copy()
df_T_2019.reset_index(inplace=True)
df_T_2019.drop(['SizeImports','index'], axis=1, inplace=True)
df_T_2019.shape

(5495, 5)

In [30]:
def split_dataframe_list(df, column):
    ''' 
    Acepta una columna y divide las variables de la lista en varias filas.
    df: dataframe para dividir
    column: la columna que contiene los valores a dividir
    '''
    
    row_accumulator = []
    
    def split_list_to_rows(row):
        split_row = row[column]
        if isinstance(split_row, list):
            for s in split_row:
                new_row = row.to_dict()
                new_row[column] = s
                row_accumulator.append(new_row)
            if split_row == []:
                new_row = row.to_dict()
                new_row[column] = None
                row_accumulator.append(new_row)
        else:
            new_row = row.to_dict()
            new_row[column] = split_row
            row_accumulator.append(new_row)
            
    df.apply(split_list_to_rows, axis=1)
    new_df = pd.DataFrame(row_accumulator)
    
    return new_df

def get_transitives_dependencies(df_anchor, df_ref, level):
    
    df_anchor = df_anchor.persist()
        
    if (level == 0): # El nivel 0 representa las dependencias directas del paquete
        df_anchor['MainPackage'] = df_anchor['Package']
        df_anchor['level'] = 0
        df_anchor['Relationship'] = df_anchor['Package'] + '|' + df_anchor['Imports']
        _df_anchor = df_anchor
        
    if (level > 0): # Dependencias transitivas
        _df_anchor = df_anchor[df_anchor.level==level]#.persist()
    
    _df = dd.merge(_df_anchor, df_ref, left_on='Imports', right_on='Package', how='inner', suffixes=('', '_x'))
    _df = _df.persist()
    
    if not(len(_df) == 0):
        print("NIVEL ", level, " SIZE ", len(_df))
        
        _df['Relationship'] =_df['Relationship'] + '|' + _df['Imports_x']
        _df['level'] = _df['level'] + 1
        _df = _df[['MainPackage', 'Package_x', 'Imports_x', 'level', 'Relationship']].rename(columns={'Package_x': 'Package', 'Imports_x': 'Imports'})#.persist()

        _df = _df.repartition(npartitions=3)
        _df = _df.reset_index(drop=True)
        
        df_anchor = df_anchor.repartition(npartitions=3)
        df_anchor = df_anchor.reset_index(drop=True)

        df_anchor_new = dd.concat([df_anchor, _df])
        
        ''''
        df_save = df_anchor_new.compute()
        df_save.to_csv("DF_ANCHOR_NEW.csv", header=df_save.columns.tolist(), sep="|", index=False)
        del df_anchor_new # Elimina variable
        
        df_an = pd.read_csv("DF_ANCHOR_NEW.csv", sep='|', low_memory=False)
        df_anchor_new = dd.from_pandas(df_an, npartitions=3)
        df_anchor_new = df_anchor_new.persist()
        '''
        return get_transitives_dependencies(df_anchor_new, df_ref, level+1)
    else:
        return df_anchor

Para obtener las relaciones entre los paquetes y sus dependencias en dos columnas se utiliza la función *split_dataframe_list()*

In [16]:
df_2010 = split_dataframe_list(df_T_2010, column='Imports')
df_2010.shape

(399, 5)

In [17]:
df_2011 = split_dataframe_list(df_T_2011, column='Imports')
df_2011.shape

(632, 5)

In [18]:
df_2012 = split_dataframe_list(df_T_2012, column='Imports')
df_2012.shape

(1140, 5)

In [19]:
df_2013 = split_dataframe_list(df_T_2013, column='Imports')
df_2013.shape

(2709, 5)

In [20]:
df_2014 = split_dataframe_list(df_T_2014, column='Imports')
df_2014.shape

(5417, 5)

In [31]:
df_2015 = split_dataframe_list(df_T_2015, column='Imports')
df_2015.shape

(10247, 5)

In [32]:
df_2016 = split_dataframe_list(df_T_2016, column='Imports')
df_2016.shape

(15884, 5)

In [33]:
df_2017 = split_dataframe_list(df_T_2017, column='Imports')
df_2017.shape

(22086, 5)

In [34]:
df_2018 = split_dataframe_list(df_T_2018, column='Imports')
df_2018.shape

(28711, 5)

In [35]:
df_2019 = split_dataframe_list(df_T_2019, column='Imports')
df_2019.shape

(33991, 5)

Utilizamos la función *get_transitives_dependencies()* para obtener la relaciones transitivas entre los paquetes

In [36]:
T_2010 = dd.from_pandas(df_2010, npartitions=3)
x_2010 = client.scatter(T_2010)
res_2010 = client.submit(get_transitives_dependencies, x_2010, x_2010, 0)
df_rel_2010 = res_2010.result().compute() # Transformamos a dataframe pandas
df_rel_2010.update(df_rel_2010.groupby('MainPackage')[['Year/Publication','Year_Month/Publication']].ffill())
df_rel_2010.shape

NIVEL  0  SIZE  258
NIVEL  1  SIZE  92
NIVEL  2  SIZE  37


(786, 8)

In [37]:
T_2011 = dd.from_pandas(df_2011, npartitions=3)
x_2011 = client.scatter(T_2011)
res_2011 = client.submit(get_transitives_dependencies, x_2011, x_2011, 0)
df_rel_2011 = res_2011.result().compute() # Transformamos a dataframe pandas
df_rel_2011.update(df_rel_2011.groupby('MainPackage')[['Year/Publication','Year_Month/Publication']].ffill())
df_rel_2011.shape

NIVEL  0  SIZE  413
NIVEL  1  SIZE  189
NIVEL  2  SIZE  97
NIVEL  3  SIZE  6


(1337, 8)

In [38]:
T_2012 = dd.from_pandas(df_2012, npartitions=3)
x_2012 = client.scatter(T_2012)
res_2012 = client.submit(get_transitives_dependencies, x_2012, x_2012, 0)
df_rel_2012 = res_2012.result().compute() # Transformamos a dataframe pandas
df_rel_2012.update(df_rel_2012.groupby('MainPackage')[['Year/Publication','Year_Month/Publication']].ffill())
df_rel_2012.shape

NIVEL  0  SIZE  840
NIVEL  1  SIZE  512
NIVEL  2  SIZE  223
NIVEL  3  SIZE  12


(2727, 8)

In [39]:
T_2013 = dd.from_pandas(df_2013, npartitions=3)
x_2013 = client.scatter(T_2013)
res_2013 = client.submit(get_transitives_dependencies, x_2013, x_2013, 0)
df_rel_2013 = res_2013.result().compute() # Transformamos a dataframe pandas
df_rel_2013.update(df_rel_2013.groupby('MainPackage')[['Year/Publication','Year_Month/Publication']].ffill())
df_rel_2013.shape

NIVEL  0  SIZE  2439
NIVEL  1  SIZE  1556
NIVEL  2  SIZE  943
NIVEL  3  SIZE  346
NIVEL  4  SIZE  85
NIVEL  5  SIZE  10


(8088, 8)

In [40]:
#res_2014 = get_transitives_dependencies(T_2014, T_2014, 0)
T_2014 = dd.from_pandas(df_2014, npartitions=3)
x_2014 = client.scatter(T_2014)
res_2014 = client.submit(get_transitives_dependencies, x_2014, x_2014, 0)
df_rel_2014 = res_2014.result().compute() # Transformamos a dataframe pandas
df_rel_2014.update(df_rel_2014.groupby('MainPackage')[['Year/Publication','Year_Month/Publication']].ffill())
df_rel_2014.shape

NIVEL  0  SIZE  7268
NIVEL  1  SIZE  5938
NIVEL  2  SIZE  3206
NIVEL  3  SIZE  1359
NIVEL  4  SIZE  405
NIVEL  5  SIZE  109
NIVEL  6  SIZE  26
NIVEL  7  SIZE  2


(23730, 8)

(23730, 8)

In [41]:
T_2015 = dd.from_pandas(df_2015, npartitions=3)
x_2015 = client.scatter(T_2015)
res_2015 = client.submit(get_transitives_dependencies, x_2015, x_2015, 0)

NIVEL  0  SIZE  17746


In [42]:
df_rel_2015 = res_2015.result().compute() # Transformamos a dataframe pandas
df_rel_2015.update(df_rel_2015.groupby('MainPackage')[['Year/Publication','Year_Month/Publication']].ffill())
df_rel_2015.shape

NIVEL  1  SIZE  16768
NIVEL  2  SIZE  10982
NIVEL  3  SIZE  6018
NIVEL  4  SIZE  2442
NIVEL  5  SIZE  867
NIVEL  6  SIZE  204
NIVEL  7  SIZE  42
NIVEL  8  SIZE  2


(65318, 8)

In [43]:
T_2016 = dd.from_pandas(df_2016, npartitions=3)
x_2016 = client.scatter(T_2016)
res_2016 = client.submit(get_transitives_dependencies, x_2016, x_2016, 0)

In [44]:
df_rel_2016 = res_2016.result().compute() # Transformamos a dataframe pandas
df_rel_2016.update(df_rel_2016.groupby('MainPackage')[['Year/Publication','Year_Month/Publication']].ffill())
df_rel_2016.shape

NIVEL  0  SIZE  32802
NIVEL  1  SIZE  40176
NIVEL  2  SIZE  30788
NIVEL  3  SIZE  17826
NIVEL  4  SIZE  6536
NIVEL  5  SIZE  2238
NIVEL  6  SIZE  540
NIVEL  7  SIZE  68


(146858, 8)

In [45]:
T_2017 = dd.from_pandas(df_2017, npartitions=3)
x_2017 = client.scatter(T_2017)
res_2017 = client.submit(get_transitives_dependencies, x_2017, x_2017, 0)

In [46]:
df_rel_2017 = res_2017.result().compute() # Transformamos a dataframe pandas
df_rel_2017.update(df_rel_2017.groupby('MainPackage')[['Year/Publication','Year_Month/Publication']].ffill())
df_rel_2017.shape

NIVEL  0  SIZE  49319
NIVEL  1  SIZE  62207
NIVEL  2  SIZE  58327
NIVEL  3  SIZE  44941
NIVEL  4  SIZE  31627
NIVEL  5  SIZE  18828
NIVEL  6  SIZE  9507
NIVEL  7  SIZE  3972
NIVEL  8  SIZE  1262
NIVEL  9  SIZE  324
NIVEL  10  SIZE  54
NIVEL  11  SIZE  4


(302458, 8)

In [47]:
T_2018 = dd.from_pandas(df_2018, npartitions=3)
x_2018 = client.scatter(T_2018) 
res_2018 = client.submit(get_transitives_dependencies, x_2018, x_2018, 0)

In [48]:
df_rel_2018 = res_2018.result().compute() # Transformamos a dataframe pandas
df_rel_2018.update(df_rel_2018.groupby('MainPackage')[['Year/Publication','Year_Month/Publication']].ffill())
df_rel_2018.shape

NIVEL  0  SIZE  82624
NIVEL  1  SIZE  126129
NIVEL  2  SIZE  138952
NIVEL  3  SIZE  112184
NIVEL  4  SIZE  75864
NIVEL  5  SIZE  45470
NIVEL  6  SIZE  21336
NIVEL  7  SIZE  8086
NIVEL  8  SIZE  2509
NIVEL  9  SIZE  688
NIVEL  10  SIZE  158
NIVEL  11  SIZE  20


(642731, 8)

In [49]:
T_2019 = dd.from_pandas(df_2019, npartitions=3)
x_2019 = client.scatter(T_2019)
res_2019 = client.submit(get_transitives_dependencies, x_2019, x_2019, 0)

In [50]:
df_rel_2019 = res_2019.result().compute() # Transformamos a dataframe pandas
df_rel_2019.update(df_rel_2019.groupby('MainPackage')[['Year/Publication','Year_Month/Publication']].ffill())
df_rel_2019.shape

NIVEL  0  SIZE  105263
NIVEL  1  SIZE  161404
NIVEL  2  SIZE  164335
NIVEL  3  SIZE  140208
NIVEL  4  SIZE  93228
NIVEL  5  SIZE  46515
NIVEL  6  SIZE  19491
NIVEL  7  SIZE  6420
NIVEL  8  SIZE  1639
NIVEL  9  SIZE  316
NIVEL  10  SIZE  30


(772840, 8)

Guarda en fichero

In [51]:
df_GT = pd.concat([df_rel_2010,df_rel_2011,df_rel_2012,df_rel_2013,df_rel_2014, df_rel_2015, df_rel_2016, df_rel_2017, df_rel_2018, df_rel_2019]).copy()
df_GT.shape

(1966873, 8)

In [52]:
df_GT.head()

,Imports,MainPackage,Package,Relationship,Version,Year/Publication,Year_Month/Publication,level
0,stats,AER,AER,AER|stats,1.1-7,2010.0,2010-10,0
1,survival,AIM,AIM,AIM|survival,1.01,2010.0,2010-04,0
2,stats,BAS,BAS,BAS|stats,0.92,2010.0,2010-10,0
3,colorspace,BiplotGUI,BiplotGUI,BiplotGUI|colorspace,0.0-6,2010.0,2010-02,0
4,KernSmooth,BiplotGUI,BiplotGUI,BiplotGUI|KernSmooth,0.0-6,2010.0,2010-02,0


In [53]:
colnames = df_GT.columns.tolist()
df_GT.to_csv("DATA_CRAN/CRAN_TRANSITIVE_RELATIONSHIPS_(2010-2019).csv", header=colnames, sep=";", index=False)